In [15]:
import random
import torch

In [457]:
'''
Parameters
self.num_players: integer number of players
self.num_cards: integer number of cards in each hand
self.current_turn: two integers, the first is the current player and the second is the turn number
self.current_suit: a character denoting the suit of the current trick in play, or None if there is no current trick
self.hands: array of integer lists, one for each hand, where zeros denote voids
self.state: integer torch tensor, where the (i, j) entry denotes the card played by the i-th player on the j-th turn
self.tricks_won: torch integer tensor, where the i-th entry is the number of tricks won by the i-th player

Methods:
self.play_card(index): plays the card of corresponding index from the current player's hand
self.get_suit(card): returns the suit of the card
self.get_value(card): returns the relative value of the card
'''
class Card_Game:

    def __init__(self, num_players, num_cards):
        self.num_cards = num_cards
        self.num_players = num_players
        self.current_turn = [0, 0]
        self.current_suit = None
        self.hands = []
        deck = [card for card in range(1, self.num_cards * self.num_players + 1)]
        random.shuffle(deck)
        for player in range(self.num_players):
            self.hands.append(deck[num_cards * player: num_cards * (player+1)])
        self.state = torch.zeros(self.num_players, self.num_cards)
        self.tricks_won = torch.zeros(self.num_players)

    '''
    Input
    index: the integer index of the card to be played from the current hand

    Description
    Replaces the card played with 0, changes the game state to reflect the played card, and changes the current turn
    '''
    def play_card(self, index):
        card = self.hands[self.current_turn[0]][index]
        # if card == 0:
        assert card != 0
        if not self.current_suit:
            self.current_suit = self.get_suit(torch.tensor(card))
        if self.current_suit != self.get_suit(torch.tensor(card)):
            for other_card in self.hands[self.current_turn[0]]:
                assert(self.get_suit(torch.tensor(other_card)) != self.current_suit)
        
        self.hands[self.current_turn[0]][index] = 0
        self.state[*self.current_turn] = card
        trick = self.state[:, self.current_turn[1]]

        if not 0.0 in trick:
            print('Trick done')
            trick_winner = self.get_trick_winner(trick)
            self.tricks_won[trick_winner] += 1
            self.current_turn[1] += 1
            self.current_turn[0] = trick_winner
        elif self.current_turn[0] == self.num_players-1:
            self.current_turn[0] = 0
        else:
            self.current_turn[0] += 1

    
    '''
    Input
    card: integer index of the card in the deck

    returns: the suit of the specified card
    '''
    def get_suit(self, card):
        suits = ['C', 'D', 'H', 'S']
        return suits[((card - 1) / float(self.num_cards)).int()]

    '''
    Input
    card: integer index of the card in the deck

    returns: the relative value of the specified card
    '''
    def get_value(self, card):
        return (card - 1) % self.num_cards

    
    '''
    Input
    trick: torch tensor of cards in the trick

    returns: the index of the player who won the trick
    '''
    def get_trick_winner(self, trick):
        trick_suit = self.get_suit(trick[0])
        card_vals = []
        for card in trick:
            if self.get_suit(card) != trick_suit:
                card_vals.append(0)
            else:
                card_vals.append(self.get_value(card))
        return card_vals.index(max(card_vals))

In [459]:
game = Card_Game(num_players=2, num_cards=12)
game.hands

[[5, 12, 17, 3, 24, 23, 2, 7, 13, 20, 18, 9],
 [4, 11, 19, 14, 21, 1, 15, 22, 8, 10, 16, 6]]

In [461]:
game.play_card(0)
game.state

tensor([[5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [463]:
game.hands

[[0, 12, 17, 3, 24, 23, 2, 7, 13, 20, 18, 9],
 [4, 11, 19, 14, 21, 1, 15, 22, 8, 10, 16, 6]]

In [465]:
game.play_card(2)
game.state

AssertionError: 

In [425]:
game.tricks_won

tensor([1., 0.])

In [427]:
game.current_turn

[0, 1]